In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.image as mpimg 
from matplotlib.pyplot import figure
import matplotlib.patches as patches

In [2]:
## Loading the data
matrices_id = pd.read_csv('../df/id_matrices.csv')

In [3]:
## list of all countries competing in Russia 2018
countries = ['Argentina','Australia','Belgium','Brazil','Colombia','Costa Rica','Croatia','Denmark','Egypt','England','France',
             'Germany','Iceland','Iran','Japan','South Korea','Mexico','Morocco','Nigeria','Panama','Peru','Poland',
             'Portugal','Russia','Saudi Arabia','Senegal','Serbia','Spain','Sweden','Switzerland','Tunisia','Uruguay']

In [4]:
matrices = []
for country in countries: 
    matrices.append(list(matrices_id[matrices_id['team_matrix']==country]['id']))

In [5]:
countries_dict = {}
index = -1
for country in countries:
    index += 1
    countries_dict[country[:]] = index

In [6]:
def xGC_PitchNet(country,matches,pattern,p=0.99,title=False,save_fig=False):
    team = []
    for matrix in matrices[countries_dict[country]]:
        team.append(pd.read_csv('{}{}'.format('../mtrx/',matrix)))
        
    for i in range(len(team)):
        team[i] = team[i].drop('zone0',1)
        team[i].index = [1,2,3,4,5,6,7,8,9]
        
    team_matches = []
    for i in matches:
        team_matches.append(team[i-1])
        
    team_sum = team_matches[0].copy()
    for i in range(1,len(team_matches)):
        team_sum = team_sum.add(team_matches[i], fill_value=0)
        
    team_bin = []
    for i in matches:
        for col in range(1,10):
            team[i-1].loc[col] = np.where(team[i-1].loc[col]>0,1,0)
        team_bin.append(team[i-1])
        
    team_sum_bin = team_bin[0]
    for i in range(1,len(team_bin)):
        team_sum_bin = team_sum_bin.add(team_bin[i], fill_value=0)

    for col in range(1,10):
        team_sum_bin.loc[col] = np.where(team_sum_bin.loc[col]>=pattern,1,0)
        
    ## 
    mtrx = np.matrix(team_sum_bin)
    mtrx = mtrx.round(decimals=3) 
    G = nx.from_numpy_matrix(mtrx,create_using = nx.MultiDiGraph())
    candidate_edges = pd.DataFrame([G.edges(data=True)]).transpose()
    candidate_edges[['node0','node1','weight']] = pd.DataFrame(candidate_edges[0].tolist(), index= candidate_edges.index)
    candidate_edges['node0'] = candidate_edges['node0']+1
    candidate_edges['node1'] = candidate_edges['node1']+1
    candidate_edges = candidate_edges.drop([0,'weight'],1)
    candidate_edges['node0'] = candidate_edges['node0'].astype('str')
    candidate_edges['node1'] = candidate_edges['node1'].astype('str')
    candidate_edges['edge'] = candidate_edges['node0']+'-'+candidate_edges['node1']
    candidate_edges = candidate_edges.drop(['node0','node1'],1)
   
    ## 
    mtrx = np.matrix(team_sum)
    mtrx = mtrx.round(decimals=3) 
    G = nx.from_numpy_matrix(mtrx,create_using = nx.MultiDiGraph())
    w = pd.DataFrame([G.edges(data=True)]).transpose()
    w[['node0','node1','weight']] = pd.DataFrame(w[0].tolist(), index= w.index)
    w['node0'] = w['node0']+1
    w['node1'] = w['node1']+1
    w = w.drop([0],1)
    w['node0'] = w['node0'].astype('str')
    w['node1'] = w['node1'].astype('str')
    w['edge'] = w['node0']+'-'+w['node1']
    w['weight'] = w['weight'].apply(lambda x: x.get('weight'))
    w['x0'] = np.where(w['node0'].isin(['1','4','7']),20,np.where(w['node0'].isin(['2','5','8']),60,100))
    w['y0'] = np.where(w['node0'].isin(['1','2','3']),9,np.where(w['node0'].isin(['4','5','6']),40,71))
    w['x1'] = np.where(w['node1'].isin(['1','4','7']),20,np.where(w['node1'].isin(['2','5','8']),60,100))
    w['y1'] = np.where(w['node1'].isin(['1','2','3']),9,np.where(w['node1'].isin(['4','5','6']),40,71))
    
    ## 
    type1 = ['1-1','2-2','3-3','4-4','5-5','6-6','7-7','8-8','9-9'] ## self-loops
    type2 = ['1-2','1-3','2-3','4-5','4-6','5-6','7-8','7-9','8-9'] ## horizontal left to right
    type3 = ['3-2','3-1','2-1','6-5','6-4','5-4','9-8','9-7','8-7'] ## horizontal rigth to left
    type4 = ['7-4','7-1','4-1','8-5','8-2','5-2','9-6','9-3','6-3'] ## vertical bottom-top
    type5 = ['1-4','1-7','4-7','2-5','2-8','5-8','3-6','3-9','6-9'] ## vertical top-bottom
    type6 = ['4-2','7-5','7-3','5-3','8-6'] ## diagonal left to right bottom-top
    type7 = ['2-4','3-5','3-7','5-7','6-8'] ## diagonal right to left top-bottom
    type8 = ['8-4','9-5','9-1','5-1','6-2'] ## diagonal right to left bottom-top
    type9 = ['4-8','1-9','1-5','5-9','2-6'] ## diagonal left to right top-bottom
    type10 = ['1-6','1-8','2-7','2-9','3-4','3-8','4-9','6-7'] ## other diagonals
    type11 = ['6-1','8-1','7-2','9-2','4-3','8-3','9-4','7-6'] ## other diagonals
    
    w['type_edge'] = np.where(w['edge'].isin(type1),'1',
                         np.where(w['edge'].isin(type2),'2',
                                 np.where(w['edge'].isin(type3),'3',
                                         np.where(w['edge'].isin(type4),'4',
                                                 np.where(w['edge'].isin(type5),'5',
                                                         np.where(w['edge'].isin(type6),'6',
                                                                  np.where(w['edge'].isin(type7),'7',
                                                                           np.where(w['edge'].isin(type8),'8','9'))))))))
    
    w['x1'] = np.where(w['type_edge']=='1',w['x1']+10,w['x1'])
    w['y1'] = np.where(w['type_edge']=='1',w['y1']-5,w['y1'])
    
    ##
    
    w['x_text'] =(w['x0']+w['x1'])/2
    w['y_text'] =(w['y0']+w['y1'])/2
    w['y_text'] = np.where(w['type_edge']=='2',w['y_text']+5,w['y_text'])
    w['y_text'] = np.where(w['type_edge']=='3',w['y_text']-5,w['y_text'])
    w['y_text'] = np.where(w['type_edge'].isin(['8']),w['y_text']-7,w['y_text'])
    w['y_text'] = np.where(w['type_edge'].isin(['9']),w['y_text']+7,w['y_text'])
    w['y_text'] = np.where(w['type_edge'].isin(['6']),w['y_text']+8,w['y_text'])
    w['y_text'] = np.where(w['type_edge'].isin(['7']),w['y_text']-8,w['y_text'])
    w['x_text'] = np.where(w['type_edge']=='4',w['x_text']+4,w['x_text'])
    w['x_text'] = np.where(w['type_edge']=='5',w['x_text']-8,w['x_text'])
    w['connectionstyle'] = np.where(w['type_edge']=='1','arc3,rad=1.5','arc3,rad=.25')
    w = w.drop(['node0','node1'],1)
    
    ##
    candidate_edges_w = candidate_edges.merge(w,on='edge',how='left')
    candidate_edges_w = candidate_edges_w.sort_values('weight',ascending=False).reset_index()
    candidate_edges_w['cumsum_weight'] = candidate_edges_w['weight'].cumsum()
    k = candidate_edges_w[candidate_edges_w['cumsum_weight']>candidate_edges_w['weight'].sum()*p].iloc[0].name + 1
    candidate_edges_w = candidate_edges_w.loc[0:k-1]
    df = candidate_edges_w.copy()
    
    ##
    x = [20,60,100,20,60,100,20,60,100]
    y = [9,9,9,40,40,40,71,71,71]
    coord = pd.DataFrame({'x':x,'y':y})
    img = mpimg.imread('../img/pitch.png') 
    plt.figure(figsize=(20,10));
    plt.xlim(0, 120)
    plt.ylim(0, 80)
    plt.gca().invert_yaxis()
    plt.imshow(img, zorder=0, extent=[0, 120, 80, 0])
    plt.axvline(40,0,c='blue')
    plt.axvline(80,0,c='blue')
    plt.axhline(18,c='blue')
    plt.axhline(62,c='blue')
    plt.scatter(x, y,c='k',s=100)
    style="Simple,tail_width=1,head_width=8,head_length=8"
    if title:
        plt.title(title,fontsize=20)
    kw = dict(arrowstyle=style, color="r")

    arrows = []
    for edge in range(len(df)):
        text = str(df['weight'][edge])
        a = patches.FancyArrowPatch((df['x0'][edge],df['y0'][edge]), (df['x1'][edge],df['y1'][edge]),connectionstyle=df['connectionstyle'][edge],**kw)
        arrows.append(a)
        plt.text(df['x_text'][edge],df['y_text'][edge], text, fontsize=15)
    

    for a in arrows:
        plt.gca().add_patch(a)
    
    if save_fig:
        plt.savefig(save_fig)
               
    return()